In [1]:
import numpy as np
import os
import sys
from stable_baselines3 import SAC
from stable_baselines3 import DQN
import torch

In [2]:
sys.path.insert(0,'boptestGym')
from boptestGymEnv import BoptestGymEnv

In [3]:
url_api = 'https://api.boptest.net'
url = "http://localhost:80"

In [4]:
import os
log_path = os.path.join( "local_files", "Logs")

# tested period: nov 1 to december 31 paper settings

In [6]:
env = BoptestGymEnv(url                   = url,
                    testcase              = 'bestest_hydronic_heat_pump',
                    actions               = ['oveTSet_u'],
                    observations          = {'reaTZon_y':(273.,324.),
                                             'weaSta_reaWeaTDryBul_y':(263.,293.),
                                             'weaSta_reaWeaHDirNor_y':(0.,862.)
                                            },
                    random_start_time     = False,
                    start_time            = 305*24*3600, #nov 1 like the paper
                    max_episode_length    = 61 * 24*3600, #2 month testing period
                    warmup_period         = 24*3600,
                    predictive_period     = 0,
                    regressive_period     = 4*1800,
                    step_period           = 1800)

C:\Users\irmak\anaconda3\Lib\site-packages\gymnasium\spaces\box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [7]:
from boptestGymEnv import NormalizedObservationWrapper
env = NormalizedObservationWrapper(env)

In [8]:
model = SAC.load("local_files/Saved Models/sac_single_temp.zip", env = env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [13]:
import requests

done = False
obs, _ = env.reset()
rows = []

from IPython.display import clear_output
i=0
while i<=2928:
    # Clear the display output at each step
    clear_output(wait=True)
    
    # Compute control signal
    action, _ = model.predict(obs, deterministic=True)
    kpis = requests.get('{0}/kpi/{1}'.format(url, env.testid)).json()['payload']

    #if isinstance(action, (tuple, list, np.ndarray)):
     #   action = int(np.array(action).flatten()[0])
    # Print the current operative temperature and decided action
    print('-------------------------------------------------------------------')
    print("obs: %s"%obs)
    print("act: %s"%action)
    print("%s /2928"%i)
    print('-------------------------------------------------------------------')
    i+=1
    # Implement action
    rows.append({
        "T_zone": obs[0],
        "t_out": obs[1],
        "Psol_Wm2": obs[2],
        "action": action,
        "energy_kWh": kpis["ener_tot"],
        "discomfort": kpis["tdis_tot"]
    })
    obs, reward, terminated, truncated, info = env.step(action)  # send the action to the environment
    done = (terminated or truncated)

-------------------------------------------------------------------
obs: [-0.11308974 -0.32333374 -1.         -0.103293   -0.09314442 -0.08384198
 -0.07960421 -0.33888954 -0.3566671  -0.36999917 -0.38333333 -1.
 -1.         -1.         -1.        ]
act: [294.76035]
2928 /2928
-------------------------------------------------------------------


In [15]:
import pandas as pd

df = pd.DataFrame(rows)

df.to_csv("local_files/testing/sac_single_temp.csv", index=False)

In [17]:
env.stop()